In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib as mpl

# style your matplotlib
mpl.style.use("seaborn-darkgrid")
# run this block

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from tqdm import tqdm

In [6]:
files=os.listdir('C:/Dell/')
files

FileNotFoundError: ignored

In [ ]:
image_array=[]  # it's a list later i will convert it to array
label_array=[]
path="../input/face-recognition-30/dataset/"
# loop through each sub-folder in train
for i in range(len(files)):
    # files in sub-folder
    file_sub=os.listdir(path+files[i])

    for k in tqdm(range(len(file_sub))):
        try:
            img=cv2.imread(path+files[i]+"/"+file_sub[k])
            img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            img=cv2.resize(img,(96,96))
            image_array.append(img)
            label_array.append(i)
        except:
            pass

100%|██████████| 319/319 [00:01<00:00, 217.40it/s]


In [ ]:
import gc
gc.collect()

458

In [ ]:

image_array=np.array(image_array)/255.0
label_array=np.array(label_array)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(image_array,label_array,test_size=0.15)

In [ ]:
from keras import layers,callbacks,utils,applications,optimizers
from keras.models import Sequential,Model,load_model

In [ ]:
len(files)

30

In [ ]:
model=Sequential()
# I will use MobileNetV2 as an pretrained model 
pretrained_model=tf.keras.applications.EfficientNetB0(input_shape=(96,96,3),include_top=False,
                                         weights="imagenet")
model.add(pretrained_model)
model.add(layers.GlobalAveragePooling2D())
# add dropout to increase accuracy by not overfitting
model.add(layers.Dropout(0.3))
# add dense layer as final output
model.add(layers.Dense(1))
model.summary()

16711680/16705208 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Functional)  (None, 3, 3, 1280)        4049571   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1281      
Total params: 4,050,852
Trainable params: 4,008,829
Non-trainable params: 42,023
_________________________________________________________________


In [ ]:
model.compile(optimizer="adam",loss="mean_squared_error",metrics=["mae"])

In [ ]:
# creating a chechpoint to save model at best accuarcy

ckp_path="trained_model/model"
model_checkpoint=tf.keras.callbacks.ModelCheckpoint(filepath=ckp_path,
                                                   monitor="val_mae",
                                                   mode="auto",
                                                   save_best_only=True,
                                                   save_weights_only=True)

In [ ]:
# create a lr reducer which decrease learning rate when accuarcy does not increase
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(factor=0.9,monitor="val_mae",
                                             mode="auto",cooldown=0,
                                             patience=5,verbose=1,min_lr=1e-6)
# patience : wait till 5 epoch
# verbose : show accuracy every 1 epoch
# min_lr=minimum learning rate
#

In [ ]:
EPOCHS=300
BATCH_SIZE=64

history=model.fit(X_train,
                 Y_train,
                 validation_data=(X_test,Y_test),
                 batch_size=BATCH_SIZE,
                 epochs=EPOCHS,
                 callbacks=[model_checkpoint,reduce_lr]
                 )

Epoch 1/300
212/212 [==============================] - 40s 120ms/step - loss: 86.7524 - mae: 7.1247 - val_loss: 188.3183 - val_mae: 11.4600
Epoch 2/300
212/212 [==============================] - 23s 106ms/step - loss: 20.0696 - mae: 3.2688 - val_loss: 94.1273 - val_mae: 7.9856
Epoch 3/300
212/212 [==============================] - 22s 105ms/step - loss: 11.8348 - mae: 2.4242 - val_loss: 67.5198 - val_mae: 6.7155
Epoch 4/300
212/212 [==============================] - 22s 105ms/step - loss: 8.8961 - mae: 2.0424 - val_loss: 67.0412 - val_mae: 6.7409
Epoch 5/300
212/212 [==============================] - 23s 108ms/step - loss: 6.6158 - mae: 1.7788 - val_loss: 66.4230 - val_mae: 6.6527
Epoch 6/300
212/212 [==============================] - 22s 105ms/step - loss: 5.5258 - mae: 1.5725 - val_loss: 140.9750 - val_mae: 9.7706
Epoch 7/300
212/212 [==============================] - 23s 106ms/step - loss: 5.5351 - mae: 1.5632 - val_loss: 66.1153 - val_mae: 6.6385
Epoch 8/300
212/212 [==============

In [ ]:
model.load_weights(ckp_path)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
prediction_val=model.predict(X_test,batch_size=BATCH_SIZE)

In [ ]:
prediction_val[:20]

array([[14.146496  ],
       [28.795204  ],
       [ 8.940104  ],
       [23.071749  ],
       [18.959055  ],
       [24.819872  ],
       [16.885336  ],
       [12.91841   ],
       [-0.07636198],
       [26.775974  ],
       [11.981905  ],
       [ 9.008382  ],
       [12.032605  ],
       [13.954588  ],
       [14.897386  ],
       [ 6.753189  ],
       [23.975328  ],
       [25.664183  ],
       [11.971088  ],
       [12.944995  ]], dtype=float32)

In [ ]:
Y_test[:20]

array([14, 29,  9, 23, 19, 25, 17, 13,  0, 27, 11,  9, 12, 14, 15, 15, 24,
       26, 12, 13])